In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import pandas as pd
import numpy as np
import statsmodels.api as sm
from tqdm import trange
from sklearn.linear_model import Ridge  
import warnings, numpy as np, pandas as pd

In [2]:
df = pd.read_csv('Trump_First_Term.csv')
df = df.sort_values(["Year", "Month", "Source"])

| Feature                        | Why it matters                                                 | How to compute                                   |
| ------------------------------ | -------------------------------------------------------------- | ------------------------------------------------ |
| `t` — running month index      | Captures secular trend across both terms.                      | `(Year-2017)*12 + Month - 1`                     |
| `sin_month`, `cos_month`       | Seasonality (cyclic month‑of‑year).                            | `sin(2π·Month/12)`, `cos(…)`                     |
| `months_into_term`             | Public opinion often evolves “within‑term”.                    | Reset to 0 at inauguration (Jan‑2017, Jan‑2025). |
| `is_election_year`             | Approval swings in election years.                             | `Year.isin({2020, 2024, 2028})`                  |
| `term` (0 = first, 1 = second) | Lets coefficients shift between terms; add interactions later. | `Year ≥ 2025`                                    |


In [3]:
df["t"] = (df["Year"] - 2017) * 12 + (df["Month"] - 1)
df["months_into_term"] = np.where(df["Year"] < 2025,
                                  df["t"],                       # first term
                                  df["t"] - ((2025-2017)*12))    # second term reset
df["sin_month"]         = np.sin(2*np.pi*df["Month"]/12)
df["cos_month"]         = np.cos(2*np.pi*df["Month"]/12)
df["is_election_year"]  = df["Year"].isin([2020, 2024, 2028]).astype(int)
df["term"]              = (df["Year"] >= 2025).astype(int)

| Feature                        | Notes                                           |
| ------------------------------ | ----------------------------------------------- |
| `approve_lag1`, `approve_lag3` | 1‑ and 3‑month lags (autoregressive signal).    |
| `approve_ma3`                  | 3‑month centred moving average (smooths noise). |
| `approve_diff1`                | Month‑to‑month change (Δ).                      |


In [4]:
df = df.sort_values(["Year", "Month", "Source"])
df["approve_lag1"] = df.groupby("Source")["Approve"].shift(1)
df["approve_ma3"]  = (df.groupby("Source")["Approve"]
                        .transform(lambda s: s.rolling(3, min_periods=1).mean()))
df["approve_diff1"] = df["Approve"] - df["approve_lag1"]

| Base series         | Expanded features                                             |
| ------------------- | ------------------------------------------------------------- |
| Unemployment rate   | Δ1, Δ12 (year‑on‑year); 3‑mo MA                               |
| Consumer sentiment  | Same transforms                                               |
| Dollar index        | Same transforms                                               |
| **Composite index** | `econ_anxiety = z(Unemp) – z(Sentiment)` (z‑score each first) |


In [5]:
macro_cols = ["Unemployment_rate",
              "Consumer_Index_Sentiment",
              "Real_Broad_Dollar_Index"]

for col in macro_cols:
    df[f"{col}_diff1"] = df[col] - df[col].shift(1)
    df[f"{col}_ma3"]   = df[col].rolling(3, min_periods=1).mean()

# Composite “economic anxiety” index
df_z = df[macro_cols].apply(lambda x: (x - x.mean())/x.std())
df["econ_anxiety"] = df_z["Unemployment_rate"] - df_z["Consumer_Index_Sentiment"]

In [6]:
df = (df.groupby(["Year","Month"])
        .apply(lambda g: g.assign(approve_consensus=g["Approve"].mean()))
        .reset_index(drop=True))
df["source_deviation"] = df["Approve"] - df["approve_consensus"]

# One‑hot for polling house
df = pd.get_dummies(df, columns=["Source"], drop_first=True)

/tmp/ipykernel_118859/1402243563.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(approve_consensus=g["Approve"].mean()))


In [7]:
train_mask = (df["Year"] < 2025) | ((df["Year"] == 2025) & (df["Month"] <= 5))
df_train   = df.loc[train_mask].copy()

feature_cols = [c for c in df.columns
                if c not in ["Approve", "Politicican", "No_Oppinion"]]

X_train = sm.add_constant(df_train[feature_cols])


y_train = df_train["Approve"]

bool_cols = X_train.select_dtypes(bool).columns
X_train[bool_cols] = X_train[bool_cols].astype(int)

# (b)  force everything else to numeric, turning non‑parsible entries into NaN
X_train = X_train.apply(pd.to_numeric, errors="coerce")
good = X_train.notna().all(axis=1) & y_train.notna()
X_train, y_train = X_train.loc[good], y_train.loc[good]
# -------------------------------------------------
# 7.  Fit GLM (use Gaussian for simplicity)
# -------------------------------------------------
glm = sm.GLM(y_train, X_train, family=sm.families.Gaussian()).fit()
ridge10 = sm.OLS(y_train, X_train).fit_regularized(alpha=10, refit=True)
print(glm.summary())

/home/tagore/miniconda3/envs/stat165/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/tagore/miniconda3/envs/stat165/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/tagore/miniconda3/envs/stat165/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


                 Generalized Linear Model Regression Results                  
Dep. Variable:                Approve   No. Observations:                  293
Model:                            GLM   Df Residuals:                      242
Model Family:                Gaussian   Df Model:                           50
Link Function:               Identity   Scale:                      3.0606e-27
Method:                          IRLS   Log-Likelihood:                 8011.6
Date:                Mon, 05 May 2025   Deviance:                   3.0506e-23
Time:                        19:47:52   Pearson chi2:                 3.05e-23
No. Iterations:                     3   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                                                coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------

In [8]:
# -------------------------------------------------
# 8.  Build the June‑2025 predictor row
# -------------------------------------------------
june_mask = (df["Year"] == 2025) & (df["Month"] == 6)

if june_mask.any():                        # June already in the CSV
    pred_df = df.loc[june_mask].copy()
else:                                      # create one from May‑2025
    pred_df = (df.loc[(df["Year"] == 2025) & (df["Month"] == 5)]
                 .iloc[[0]]                # keep as DataFrame
                 .copy())

    # --- bump time features ---
    pred_df["Month"] = 6
    pred_df["t"]    += 1
    pred_df["months_into_term"] += 1
    pred_df["sin_month"] = np.sin(2*np.pi*6/12)
    pred_df["cos_month"] = np.cos(2*np.pi*6/12)

    # --- OPTIONAL: plug in macro forecasts ---
    # pred_df["Unemployment_rate"]        = 4.0
    # pred_df["Consumer_Index_Sentiment"] = 85.0
    # pred_df["Real_Broad_Dollar_Index"]  = 104.0

    # refresh diff / MA transforms
    for col in macro_cols:
        pred_df[f"{col}_diff1"] = pred_df[col] - df[col].iloc[-1]
        pred_df[f"{col}_ma3"]   = pd.concat([df[col].tail(2),
                                             pred_df[col]]).mean()

    # recompute composite index
    pred_df["econ_anxiety"] = (
        (pred_df["Unemployment_rate"] - df["Unemployment_rate"].mean())
        / df["Unemployment_rate"].std()
        -
        (pred_df["Consumer_Index_Sentiment"] - df["Consumer_Index_Sentiment"].mean())
        / df["Consumer_Index_Sentiment"].std()
    )

    # lagged approval placeholders (optional fine‑tuning)
    pred_df["approve_lag1"]  = pred_df["Approve"]        # assumes flat
    pred_df["approve_diff1"] = 0
    pred_df["approve_ma3"]   = pred_df["Approve"]

# -------------------------------------------------
# 9.  Pollster consensus / deviation for June
# -------------------------------------------------
pred_df["approve_consensus"] = pred_df.groupby(["Year","Month"])["Approve"].transform("mean")
pred_df["source_deviation"]  = pred_df["Approve"] - pred_df["approve_consensus"]

# -------------------------------------------------
# 10.  Re‑build Source_* dummies so columns match X_train
# -------------------------------------------------
for col in [c for c in X_train.columns if c.startswith("Source_")]:
    pollster = col.replace("Source_", "")
    if col not in pred_df.columns:               # column was dropped earlier
        pred_df[col] = 0
    if "Source" in pred_df.columns:              # normal case
        pred_df[col] = (pred_df["Source"] == pollster).astype(int)

# -------------------------------------------------
# 11.  Final column alignment & coercion
# -------------------------------------------------
X_june = (pred_df.reindex(columns=X_train.columns.drop("const"), fill_value=0)
                    .apply(pd.to_numeric, errors="coerce")
                    .fillna(0))

X_june = sm.add_constant(X_june, has_constant="add")
X_june = X_june.reindex(columns=X_train.columns, fill_value=0)

# -------------------------------------------------
# 12.  Predict June‑2025 approval
# -------------------------------------------------
pred = glm.predict(X_june)
print(f"\nPredicted Trump approval (Gallup‑style) for June 2025: {pred.iloc[0]:.2f}%")


Predicted Trump approval (Gallup‑style) for June 2025: 45.00%


In [9]:


B      = 500           # bootstrap replications
alpha  = 0.10          # 80 % CI  ⇒ 10th & 90th pct.
lambda_ = 10           # same ridge penalty

months = df_train[["Year", "Month"]].drop_duplicates().values
boot_preds = []

for _ in trange(B):
    # --- resample months with replacement ---
    rows = pd.concat([df_train[(df_train["Year"] == y) & (df_train["Month"] == m)]
                      for y, m in months[np.random.choice(len(months),
                                                           size=len(months),
                                                           replace=True)]],
                     ignore_index=True)

    Xb = rows[feature_cols]; yb = rows["Approve"]
    Xb = sm.add_constant(Xb).apply(pd.to_numeric, errors="coerce").fillna(0)

    mdl = Ridge(alpha=lambda_, fit_intercept=False).fit(Xb.values, yb.values)
    boot_preds.append(mdl.predict(X_june.values)[0])

boot_preds = np.array(boot_preds)
point = boot_preds.mean()
lo, hi = np.percentile(boot_preds, [100*alpha/2, 100*(1-alpha/2)])

print(f"June‑2025 approval (ridge): {point:.2f}%  — 80 % bootstrap CI: {lo:.2f} – {hi:.2f}%")


100%|██████████| 500/500 [00:25<00:00, 19.65it/s]

June‑2025 approval (ridge): 44.97%  — 80 % bootstrap CI: 44.91 – 45.03%


Thought for a couple of seconds


**High‑level idea of the bootstrap script**

1. **Choose settings**

* `B = 500` → run 500 bootstrap replications.
* `alpha = 0.20` → we want an 80 % confidence interval (the 10th and 90th percentiles).
* `ridge = 5.0` → add a small L2 penalty so each refit can’t perfectly over‑fit the data.

2. **Resample the data 500 times**

* Treat *one calendar month* as the atomic unit.
* For every replication, draw the same number of months as in the original sample, **with replacement**.
* Collect every poll that belongs to each chosen month to build a new dataset.

3. **Refit the model on each resampled dataset**

* Re‑create the design matrix (`Xb`) and target vector (`yb`).
* Drop any rows with missing numbers.
* Fit either a ridge regression (if `ridge > 0`) or an ordinary OLS.
* Use that model to predict Trump’s approval for June 2025 (`X_june`).
* Store the prediction in `boot_preds`.

4. **Convert the 500 predictions into an interval**

* Mean of the 500 numbers → bootstrap point estimate.
* 10th percentile → lower bound of the 80 % CI.
* 90th percentile → upper bound.

5. **Print the result**

```
Bootstrap point estimate (mean of 500 reps):  <point> %
80% bootstrap CI: [<lower> %, <upper> %]
```

**Why it works**
The month‑level resampling makes coefficients and predictions wobble in a way that mimics real‑world sampling variability, so the spread of the 500 forecasts becomes a non‑parametric confidence interval around the June‑2025 prediction.


In [10]:
B      = 500      # number of bootstrap replications
alpha  = 0.20     # 80% CI  ⇒  10th & 90th percentiles
ridge  = 5.0      # small λ to avoid perfect fit; set 0 for plain OLS

# 1) list of unique (Year,Month) in the training window
months = df_train[["Year", "Month"]].drop_duplicates().values

boot_preds = []

for _ in trange(B, desc="Bootstrapping"):
    # 2) resample months with replacement
    sample_idx = np.random.choice(len(months), size=len(months), replace=True)
    boot_rows  = pd.concat(
        [df_train.loc[(df_train["Year"] == months[i][0]) &
                      (df_train["Month"] == months[i][1])]
         for i in sample_idx],
        ignore_index=True)

    # 3) rebuild X / y for this bootstrap
    Xb = sm.add_constant(boot_rows[feature_cols])
    yb = boot_rows["Approve"]

    # numerics only, drop rows with any NaN
    bool_cols = Xb.select_dtypes(bool).columns
    Xb[bool_cols] = Xb[bool_cols].astype(int)
    Xb = Xb.apply(pd.to_numeric, errors="coerce")
    good = Xb.notna().all(axis=1) & yb.notna()
    Xb, yb = Xb.loc[good], yb.loc[good]

    # 4) fit (Ridge ≈ OLS if ridge=0)
    if ridge > 0:
        mdl = Ridge(alpha=ridge, fit_intercept=False)   # const already in Xb
        mdl.fit(Xb.values, yb.values)
        pred = mdl.predict(X_june.values)[0]
    else:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")             # ignore cond num warnings
            mdl = sm.OLS(yb, Xb).fit()
        pred = mdl.predict(X_june)[0]

    boot_preds.append(pred)

boot_preds = np.array(boot_preds)
lower = np.percentile(boot_preds, 100*alpha/2)
upper = np.percentile(boot_preds, 100*(1-alpha/2))
point = boot_preds.mean()

print(f"\nBootstrap point estimate (mean of {B} reps): {point:5.2f}%")
print(f"80% bootstrap CI: [{lower:5.2f}%, {upper:5.2f}%]")


Bootstrapping: 100%|██████████| 500/500 [00:28<00:00, 17.50it/s]


Bootstrap point estimate (mean of 500 reps): 44.99%
80% bootstrap CI: [44.98%, 45.00%]


Thought for a couple of seconds


### High‑level overview of the block‑bootstrap forecasting code

1. **Goal**
   Produce a *realistic* 80 % prediction interval (PI) for Donald Trump’s Gallup approval in **June 2025**, accounting for

   * parameter uncertainty,
   * short‑term autocorrelation in monthly data, and
   * Gallup’s own sampling error.

2. **Key ingredients**

   | Symbol             | Meaning                                                                     |
   | ------------------ | --------------------------------------------------------------------------- |
   | `B = 1000`         | number of bootstrap replications                                            |
   | `block_k = 3`      | resample **3‑month blocks** to keep local time‑dependence intact            |
   | `λ = 10`           | ridge penalty that prevents over‑fitting inside each resample               |
   | `poll_se = 1.5 pp` | extra ±1.5 percentage‑point noise reflecting Gallup’s survey sampling error |

3. **Workflow per replication**

   1. **Resample months** — draw random starting indices and glue together 3‑month blocks until you rebuild a full history (with replacement).
   2. **Refit model** — run a ridge regression on that resampled dataset.
   3. **Predict June 2025** using the fitted ridge coefficients.
   4. **Add survey noise** — jitter the prediction by one random draw from 𝒩(0, poll\_se²).
   5. **Store the result** in the list `boot_pred`.

4. **After all replications**

   * Convert `boot_pred` → NumPy array.
   * **Point estimate** = mean of the 1 000 predictions.
   * **80 % PI** = 10th and 90th percentiles of that distribution.
   * Print:

     ```
     June‑2025 Gallup approval: <mean>%   (80% PI: <lo>% – <hi>%)
     ```

5. **Why it matters**
   *Moving‑block bootstrapping* captures serial correlation that plain IID resampling would miss.
   **Ridge** keeps the model stable when a resample omits rare pollster dummies.
   **Sampling‑error noise** widens the band to honor the fact that each Gallup poll is itself a sample.

The end result is a prediction interval that reflects *all three* major uncertainty sources—model fit, month‑to‑month shocks, and polling error—yielding a far more credible range than the vanishing‑width interval produced by an over‑fit OLS model.


In [ ]:
B        = 1000        # replications
alpha    = 0.10        # 80% PI
lambda_  = 10         # ridge penalty
block_k  = 15          # 3‑month moving blocks
poll_se  = 3         # sampling error of a single Gallup poll (pp)

months   = df_train[["Year","Month"]].drop_duplicates().values
boot_pred = []

for _ in trange(B):
    # -- draw starting indices for moving blocks
    starts = np.random.randint(0, len(months)-block_k+1,
                               size=len(months)//block_k + 1)
    rows = pd.concat([df_train[(df_train["Year"]==months[i+j][0]) &
                               (df_train["Month"]==months[i+j][1])]
                      for i in starts for j in range(block_k)],
                     ignore_index=True)

    Xb = sm.add_constant(rows[feature_cols]).apply(pd.to_numeric, errors='coerce').fillna(0)
    yb = rows["Approve"]

    mdl = Ridge(alpha=lambda_, fit_intercept=False).fit(Xb.values, yb.values)
    mu  = mdl.predict(X_june.values)[0]

    # add Gallup sampling error
    mu += np.random.normal(scale=poll_se)

    boot_pred.append(mu)

# centre & percentiles
boot_pred = np.array(boot_pred)
pt   = boot_pred.mean()
lo, hi = np.percentile(boot_pred, [100*alpha/2, 100*(1-alpha/2)])

print(f"June‑2025 Gallup approval: {pt:4.1f}%   (80% PI: {lo:4.1f} – {hi:4.1f}%)")


  0%|          | 0/1000 [00:00<?, ?it/s]

 17%|█▋        | 174/1000 [00:09<00:43, 19.15it/s]